# Combinatorics of Hurwitz numbers

Author: [Victoria Schleis](https://victoriaschleis.github.io/)

Based on: Agostini, Markwig, Nollau, Schleis, Sendra, Sturmfels: [Recovery of plane curves from branch points](https://arxiv.org/abs/2205.11287), arXiv:2205.11287.


This notebook describes the OSCAR <font color=blue>[Oscar]</font> code used to prove some of the results in the paper above, for the smaller of the two cases discussed. The code necessary for further computations in the larger case can be found on the [mathrepo](https://mathrepo.mis.mpg.de/BranchPoints/monodromy.html). 

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


## Theoretical Background 

Enumerative algebraic geometry is a subfield of algebraic geometry concerned with counting invariants. Enumerative problems include the classical problem of counting how many curves of degree $d$ pass through a collection of points in some space or algebraic variety. 

### Counting maps of Riemann surfaces: Hurwitz numbers

Here, we will count meromorphic maps of compact complex manifolds of dimension 1, called _Riemann Surfaces_ to the projective line $\mathbb{P}^1$, $f:X\rightarrow \mathbb{P}^1$. 

We fix the genus $g$ of $X$ and the degree $d$ of $f$. For this notebook, we will consider Riemann surfaces of genus 1 and maps of degree 3, in our paper  <font color=blue>[AMNSSS]</font> we further cover the case of genus 3 and degree 4. Maps of Riemann surfaces have a special structure: Outside of a finite set of points (in our case six) called _branch points_, maps of Riemann surfaces are (topological) covers of degree $d$.

Thus, outside of the branch points, every point has exactly $d$ preimages, whereas the branch points have less preimages. Since the maps we are considering are meromorphic maps of complex manifolds, locally they can be described as $f=z^k$ around the preimages of points on $\mathbb{P}^1$. Now, we impose constraints on the $k$ in the local representation: In our concrete problem, we fix the branch points $x_1,\dots,x_6$ and we require the map to have _simple ramification_ over all $x_i$, i.e. every $x_i$ has two preimages $y_i$ and $w_i$, and the local expression of the holomorphic map is given by $f = z^2$ around one of them, and by $f = z$ around the other. 


### Monodromy representations

Monodromy representations allow us to view the geometric situation described above in a group-theoretic and combinatorial light: The behaviour of the cover around the preimage branch points can be described in terms of permutations by investigating preimages of loops around branch points, associating a transposition to each loop. For details, see <font color=blue>[CM]</font>.

Explicitely, a monodromy representation of degree $d$ is a list of permutations $(\tau_1, \dots, \tau_i)$ in $\mathbb{S}_d$ such that the $\tau_i$ act transitively on $(1,\dots,d)$ and $\tau_d\circ\cdots\circ \tau_1 = id$, and monodromy representations are in 1-1 correspondence with covers.  

For our project, we implemented utility in OSCAR <font color=blue>[Oscar]</font> for constructing and analyzing monodromy representations. For instance, we can check whether a given vector of permutations acts transitively on $(1,2,3)$:

In [2]:
function is_monodromy_representation_degree_3(monod::Vector{Perm{Int64}})
    for i in 1:length(monod)
        if monod[i] == Perm([3,2,1]) || monod[i] ==Perm([1,3,2])
            return true
        end
    end
    return false
end


is_monodromy_representation_degree_3([Perm([2,1,3]),Perm([3,2,1]),Perm([3,2,1]),Perm([3,2,1])])

true

The goal of the remainder of this section will be to construct a complete list of monodromy representations of degree 3 and genus 1, using OSCAR <font color=blue>[Oscar]</font> methods for permutations. 

For this, we will need to count monodromy representations by counting equivalence classes under conjugation by the symmetric group $\mathbb{S}_3$. We say that the canonical representative of such a class is the lexicographically smallest element, using $(1,2)<(1,3)<(2,3)$. The following function ``canonical_monodromy_representation_degree_3`` takes a monodromy representation of degree 3 and returns the canonical representative under this group action, using two auxiliary functions ``order_number`` and ``swap``. 

``order_number`` returns the index of a permutation in the ``order_list``. This is necessary to order our permutations lexicographically:

In [3]:
function order_number(a::Perm{Int64}, order_list::Vector{Perm{Int64}})
    count = 1
    for i in 1:length(order_list)
        if order_list[i] == a
            return count
        end
        count+=1
    end
end
transpos_order=[Perm([2,1,3]),Perm([3,2,1]),Perm([1,3,2])]
println(order_number(Perm([3,2,1]), transpos_order))

2


``swap`` swaps all occurences of the two permutations $a$ and $b$ within the monodromy list. 

In [4]:
function swap(monod::Vector{Perm{Int64}},a::Perm{Int64},b::Perm{Int64})
    for i in 1:length(monod)        
        if monod[i] == a
            monod[i] = b
        elseif monod[i] == b
            monod[i] = a
        end
    end    
    return monod
end

swap (generic function with 1 method)

Now, ``canonical_monodromy_representation_degree_3`` computes a canonical representative in the lexicographical ordering by swapping elements of the array successively so that they appear in lexicographical order, while preserving the group orbit.

In [5]:
function canonical_monodromy_representation_degree_3(monod::Vector{Perm{Int64}})
    transpos_order=[Perm([2,1,3]),Perm([3,2,1]),Perm([1,3,2])]
    current_index = 1
    m = deepcopy(monod)
    for i in 1:length(m)
        if order_number(m[i],transpos_order)>current_index
            m = swap(m,transpos_order[current_index],m[i])
            current_index+=1
        elseif order_number(m[i],transpos_order)==current_index
            current_index+=1
        end
    end
    return m
end
# Already a canonical representative
v= [Perm([2,1,3]),Perm([3,2,1]),Perm([3,2,1]),Perm([3,2,1]),Perm([3,2,1]),Perm([2,1,3])]
println(canonical_monodromy_representation_degree_3(v))

# Not a canonical representative
w= [Perm([3,2,1]),Perm([2,1,3]),Perm([3,2,1]),Perm([3,2,1]),Perm([3,2,1]),Perm([2,1,3])]
println(canonical_monodromy_representation_degree_3(w))

Perm{Int64}[(1,2), (1,3), (1,3), (1,3), (1,3), (1,2)]
Perm{Int64}[(1,2), (1,3), (1,2), (1,2), (1,2), (1,3)]


We can use this function now to recursively generate all monodromy representations of degree $3$ using the function ``all_monodromy_reps_degree_3``. 

``all_monodromy_reps_degree_3`` works by recursively generating monodromy representations. For this, we take an array of permutations of length shorter than the monodromy representation and add new permutations to it, following the lexicographical order. Recall that if an array of permutations $(\tau_1, \dots, \tau_d)$ is a monodromy representation, then $\tau_d\circ\cdots\circ \tau_1 = id$. Thus, at length $d-1$, the permutation we need to append needs to be $(\tau_{d-1}\circ\cdots\circ \tau_1)^{-1}$. This is what ``monodromy_length_minus_1`` ensures.

In [6]:
function monodromy_length_minus_1(already_constructed::Vector{Perm{Int64}})
    m = already_constructed[1]
    for i in 2:length(already_constructed)
        m = m*already_constructed[i]
    end
    c = permtype(m)
    if c[1] == 2
        if c[2] == 1
            n = [inv(m)]
                return (true,cat(already_constructed,n,dims=1))
            end
        end
    return (false,Perm([1,2,3]))             
end

monodromy_length_minus_1 (generic function with 1 method)

In ``all_monodromy_reps_internal_degree_3``, we do the main part of the generation of monodromy representations. We recursively add permutations to the array in an ordered way, using upper bounds for which permutations can appear in a canonical representative. If we achieve length $d-1$, we call ``monodromy_length_minus_1`` and construct an appropriate candidate, which we check for being transitive and canonical using the methods described above. If the vector fulfills all these requirements, we have constructed a monodromy representation and add it to the list ``already_constructed``.

In [7]:
function all_monodromy_reps_internal_degree_3(already_constructed::Vector{Perm{Int64}}, transpos::Vector{Perm{Int64}},current_length::Int64, wanted_length::Int64, full_monodromy::Vector{Vector{Perm{Int64}}})
    upper_bound = min(current_length+1,length(transpos))
    for i in 1:upper_bound
        already_constructed_temp =deepcopy(already_constructed)
        push!(already_constructed_temp,transpos[i])
        current_length = length(already_constructed_temp)
        if current_length == wanted_length-1
            out = monodromy_length_minus_1(already_constructed_temp)
            if out[1] && is_monodromy_representation_degree_3(out[2]) && out[2] == canonical_monodromy_representation_degree_3(out[2])
                push!(full_monodromy,out[2])
            end
        elseif current_length > wanted_length-1
            return "ERROR"
        else
            all_monodromy_reps_internal_degree_3(already_constructed_temp, transpos,current_length, wanted_length, full_monodromy)
        end
    end
end

all_monodromy_reps_internal_degree_3 (generic function with 1 method)

Since all monodromy representations are counted by their canonical representatives, we can assume the first transposition of each monodromy representation to be $(1,2)$. Now, we use the two functions described above and collect their results in a list. 

In [8]:
function all_monodromy_reps_degree_3()
    Transpositions = [Perm([2,1,3]),Perm([3,2,1]),Perm([1,3,2])]
    already_constructed=[Perm([2,1,3])]
    full_monodromy = Vector{Vector{Perm{Int64}}}([])
    all_monodromy_reps_internal_degree_3(already_constructed,Transpositions,1,6,full_monodromy)
    return full_monodromy
end
l = all_monodromy_reps_degree_3()

40-element Vector{Vector{Perm{Int64}}}:
 [(1,2), (1,2), (1,2), (1,2), (1,3), (1,3)]
 [(1,2), (1,2), (1,2), (1,3), (1,2), (2,3)]
 [(1,2), (1,2), (1,2), (1,3), (1,3), (1,2)]
 [(1,2), (1,2), (1,2), (1,3), (2,3), (1,3)]
 [(1,2), (1,2), (1,3), (1,2), (1,2), (1,3)]
 [(1,2), (1,2), (1,3), (1,2), (1,3), (2,3)]
 [(1,2), (1,2), (1,3), (1,2), (2,3), (1,2)]
 [(1,2), (1,2), (1,3), (1,3), (1,2), (1,2)]
 [(1,2), (1,2), (1,3), (1,3), (1,3), (1,3)]
 [(1,2), (1,2), (1,3), (1,3), (2,3), (2,3)]
 [(1,2), (1,2), (1,3), (2,3), (1,2), (2,3)]
 [(1,2), (1,2), (1,3), (2,3), (1,3), (1,2)]
 [(1,2), (1,2), (1,3), (2,3), (2,3), (1,3)]
 ⋮
 [(1,2), (1,3), (1,3), (2,3), (1,2), (1,3)]
 [(1,2), (1,3), (1,3), (2,3), (1,3), (2,3)]
 [(1,2), (1,3), (1,3), (2,3), (2,3), (1,2)]
 [(1,2), (1,3), (2,3), (1,2), (1,2), (1,3)]
 [(1,2), (1,3), (2,3), (1,2), (1,3), (2,3)]
 [(1,2), (1,3), (2,3), (1,2), (2,3), (1,2)]
 [(1,2), (1,3), (2,3), (1,3), (1,2), (1,2)]
 [(1,2), (1,3), (2,3), (1,3), (1,3), (1,3)]
 [(1,2), (1,3), (2,3), (1,3), (2,

## Bibliography

<font color=blue>[AMNSSS]</font> Agostini, Markwig, Nollau, Schleis, Sendra, Sturmfels: [Recovery of plane curves from branch points](https://arxiv.org/abs/2205.11287). arXiv:2205.11287 (2022)

<font color=blue>[CM]</font> Cavalieri, Miles. Riemann Surfaces and Algebraic Curves: A First Course in Hurwitz Theory. Cambridge University Press (2016). doi:10.1017/CBO9781316569252

<font color=blue>[Oscar]</font>
    OSCAR -- Open Source Computer Algebra Research system, Version 0.12.0 The OSCAR Team, (2023). (https://www.oscar-system.org)